<div align = "center">

# **Exploración de datos (Base siniestros)**

</div>

## Librerias

In [ ]:
library(readxl)
library(dplyr)

## Data

In [ ]:
Siniestros <- read_excel("../data/input/Siniestros_Hist.xlsx")

In [ ]:
# Ver nombres actuales del dataframe Siniestros
names(Siniestros)

In [ ]:
# Método directo para renombrar columnas
names(Siniestros) <- c(
  "Fecha_siniestro",
  "Prima_efectivamente_pagada_hasta_fecha_siniestro",
  "Prima_suscrita_incluida_cobertura_responsabilidad_civil",
  "Asegurado_otras_coberturas_distintas_responsabilidad_civil",
  "Asegurado_original",
  "Efectivamente_pagado_por_siniestro",
  "Siniestro_incurrido",
  "Deducible",
  "Recobro_obtenido",
  "Reserva_constituida_para_amparo",
  "Reserva_pagada_para_amparo",
  "Fecha_pago_amparo",
  "Estado_final_siniestro",
  "Cobertura_final_aplicada"
)

In [ ]:
unique(Siniestros$Cobertura_final_aplicada)

In [ ]:
# Cambiar los niveles de la columna Cobertura_final_aplicada
Siniestros$Cobertura_final_aplicada <- factor(Siniestros$Cobertura_final_aplicada, 
                                           levels = c("PTH", "PPD", "RC BIENES", "PPH", "RC PERS"),
                                           labels = c("Perdida_total_hurto", 
                                                     "Perdida_parcial_danos",
                                                     "Responsabilidad_civil_bienes",
                                                     "Perdida_parcial_hurto",
                                                     "Responsabilidad_civil_personas"))

In [ ]:
# Verigficar los cambios
levels(Siniestros$Cobertura_final_aplicada)

In [ ]:
head(Siniestros)

In [ ]:
# Primero corregir el problema de variables - usar Siniestros consistentemente
# Crear columnas de fecha
Siniestros$Año <- format(Siniestros$Fecha_siniestro, "%Y")
Siniestros$Mes <- format(Siniestros$Fecha_siniestro, "%m")
Siniestros$Día <- format(Siniestros$Fecha_siniestro, "%d")

# Verificar los tipos de cobertura únicos
print("Tipos de cobertura únicos:")
print(unique(Siniestros$Cobertura_final_aplicada))

In [ ]:
## Limpieza de los datos con los que se trabajan inicialmente

### 1. en las celdas anteriores, se renomnraron tanto las columnas, como el contenido de la columna Cobertura_final_aplicada
### 2. ahora, vamos a filtrar las columnas que noc coinciden según los dataframes de polizas vendidad y polizas siniestradas, que son solamente 2018


In [42]:
# Filtrar los datos para incluir solo los siniestros de 2018
Siniestros <- Siniestros %>%
    filter(format(Fecha_siniestro, "%Y") == "2018")

# Verificar los resultados
print(min(Siniestros$Fecha_siniestro))
print(max(Siniestros$Fecha_siniestro))

[1] "2018-01-01 UTC"


[1] "2018-12-31 UTC"


In [ ]:
# Tabla 1: Cantidad de siniestros por fecha y tipo de cobertura

## Fecha de siniestro
## Siniestro incurrido

library(tidyr)

tabla_cantidad <- Siniestros %>%
  filter(!is.na(Cobertura_final_aplicada)) %>%  # Excluir NA
  group_by(Año, Mes, Día, Cobertura_final_aplicada, Siniestro_incurrido) %>%
  summarise(Cantidad_siniestros = n(), .groups = 'drop') %>%
  pivot_wider(names_from = Cobertura_final_aplicada, 
              values_from = Cantidad_siniestros, 
              values_fill = 0)

print("Tabla 1: Cantidad de siniestros por fecha y tipo de cobertura")
head(tabla_cantidad)

In [ ]:
# Tabla 2: Valor efectivamente pagado por fecha y tipo de cobertura
tabla_valor <- Siniestros %>%
  filter(!is.na(Cobertura_final_aplicada)) %>%  # Excluir NA
  group_by(Año, Mes, Día, Cobertura_final_aplicada) %>%
  summarise(Valor_pagado = sum(Efectivamente_pagado_por_siniestro, na.rm = TRUE), .groups = 'drop') %>%
  pivot_wider(names_from = Cobertura_final_aplicada, 
              values_from = Valor_pagado, 
              values_fill = 0)

print("Tabla 2: Valor efectivamente pagado por fecha y tipo de cobertura")
head(tabla_valor)

In [ ]:
# Verificar las dimensiones de las tablas
print(paste("Dimensiones tabla cantidad:", dim(tabla_cantidad)[1], "x", dim(tabla_cantidad)[2]))
print(paste("Dimensiones tabla valor:", dim(tabla_valor)[1], "x", dim(tabla_valor)[2]))

# Mostrar resumen de las tablas
print("\nResumen tabla cantidad:")
summary(tabla_cantidad)

print("\nResumen tabla valor:")
summary(tabla_valor)

In [ ]:
# Guardar las tablas pivoteadas como archivos CSV
write.csv(tabla_cantidad, "../data/processed/tabla_cantidad.csv", row.names = FALSE)
write.csv(tabla_valor, "../data/processed/tabla_valor.csv", row.names = FALSE)

print("Tablas guardadas exitosamente:")
print("- tabla_cantidad.csv en data/processed/")
print("- tabla_valor.csv en data/processed/")